# Homework 2 - Considering Bias in Data

## Pre Processing for Analysis

## License
This code example was developed by Aaditya Parthasarathy for a homework assignment in DATA 512, a course in the UW MS Data Science degree program. Some of the code is partially adopted from the jupyter notebook developed by Dr. David W. McDonald, which is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 14, 2023

### Step 1: Getting the data and initial cleanup

We take our first step in importing the articles and cleaning it up by removing duplicates

In [1]:
# Import statements
import pandas as pd
import json
import time

import aiohttp
import asyncio

Let us import the article list, which is in the data folder as us_cities_by_state_SEPT.2023.csv for cleanup

In [107]:
#fetching the article list with states and cities
us_cities_by_state = pd.read_csv('data/us_cities_by_state_SEPT.2023.csv')

# Dropping duplicates due to inconsistencies with the state list
us_cities_by_state.drop_duplicates(inplace=True, ignore_index=True)

In [110]:
#checking to see if we missed any unwanted entries
a = list(us_cities_by_state.page_title)
b = {}
c = set()

for state in a:
    if state in b:
        c.add(state)
    else:
        b[state] = 1
us_cities_by_state[us_cities_by_state['page_title'].isin(c)]

,state,page_title,url
1683,Colorado,2020 United States census,https://en.wikipedia.org/wiki/2020_United_Stat...
1684,Colorado,2010 United States census,https://en.wikipedia.org/wiki/2010_United_Stat...
2030,Florida,County (United States),https://en.wikipedia.org/wiki/County_(United_S...
5196,Iowa,County (United States),https://en.wikipedia.org/wiki/County_(United_S...
12947,New_York,Population,https://en.wikipedia.org/wiki/Population
18433,Tennessee,County (United States),https://en.wikipedia.org/wiki/County_(United_S...
18780,Texas,Population,https://en.wikipedia.org/wiki/Population
18781,Texas,2020 United States census,https://en.wikipedia.org/wiki/2020_United_Stat...
18782,Texas,2010 United States census,https://en.wikipedia.org/wiki/2010_United_Stat...
21234,Wisconsin,County (United States),https://en.wikipedia.org/wiki/County_(United_S...


We now need to remove these rows from the original dataframe

In [111]:
us_cities_by_state.drop(us_cities_by_state[us_cities_by_state['page_title'].isin(c)].index, inplace=True)


In [112]:
us_cities_by_state

,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"
...,...,...,...
21520,Wyoming,"Wamsutter, Wyoming","https://en.wikipedia.org/wiki/Wamsutter,_Wyoming"
21521,Wyoming,"Wheatland, Wyoming","https://en.wikipedia.org/wiki/Wheatland,_Wyoming"
21522,Wyoming,"Worland, Wyoming","https://en.wikipedia.org/wiki/Worland,_Wyoming"
21523,Wyoming,"Wright, Wyoming","https://en.wikipedia.org/wiki/Wright,_Wyoming"


This is now ready for processing. We can note here that we have 21515 articles/places for which we fetch the information of last revision id and the quality scores. This would be a helpful number to check and keep track of

### Step 2: Getting last revision id for an article



This code is based on a Jupyter notebook originally developed by Dr. David W. McDonald. I have adapted it to use asynchronous functions, allowing for concurrent requests and faster processing. It's important to be aware that frequent or intensive use of this code may lead to temporary or permanent API access restrictions, so exercise caution. You can view the original code in helper_code/wp_page_info_example.ipynb

Additionally, please remember to configure the REQUEST_HEADERS variable with your email address. Be sure to review the API documentation for information on rate limits and adjust this code as needed to suit your specific use case.

In [113]:

### Define constants
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_THROTTLE_WAIT = 0.1

# Define other constants and variables
REQUEST_HEADERS = {
    'User-Agent': '<aadi2000@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"

PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",  # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}
       
# this is an async function to create and pull multiple requests from the API.
async def request_pageinfo_per_article(session, article_title, endpoint_url=API_ENWIKIPEDIA_ENDPOINT,
                                       request_template=PAGEINFO_PARAMS_TEMPLATE, headers=REQUEST_HEADERS):
    request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    async with session.get(endpoint_url, headers=headers, params=request_template) as response:
        try:
            json_response = await response.json()
            # Get the page ID for the article
            page_id = list(json_response['query']['pages'].keys())[0]
            lastrevid = json_response['query']['pages'][page_id]['lastrevid']
            return lastrevid
        except Exception as e:
            print(f"Failed to retrieve data for {article_title}: {e}")
            return None

# This is the main function, where we create multiple sessions, and fetch the last revision id
async def main(article_titles, page_info_dict, unable_to_get):
    async with aiohttp.ClientSession() as session:
        tasks = []
        count = 0  # Initialize a counter
        for i, title in enumerate(article_titles):
            task = request_pageinfo_per_article(session, title)
            tasks.append(task)
            if i % 10 == 0:
                print(f"{i} of {len(article_titles)} have been requested")
                
            if (i + 1) % 1000 == 0:
                print(f"{i + 1} articles' IDs have been stored")
                count += 1
        responses = await asyncio.gather(*tasks)

        #once we get the scores, we can look at and store the ids in a dictionary
        for i, response in enumerate(responses):
            if response is not None:
                page_info_dict[article_titles[i]] = response
            else:
                print(f"Failed to retrieve data for {article_titles[i]}")
                unable_to_get[article_titles[i]] = 1
        print(f"{count * 1000} articles are done")
    return page_info_dict, unable_to_get

# Define the list of articles
article_list = list(us_cities_by_state["page_title"])


As we perform it asycronously, there are chances of it stopping in the middle. to save the processed information, we take a batch size of 1000, and save for every 1000 articles performed

In [ ]:
# Define the batch size
batch_size = 1000
batch_number = 0

# Create a list to store the results of each batch
results = []

# Iterate over the article_list in batches of size batch_size
for i in range(0, len(article_list), batch_size):
    batch_number += 1
    article_list1 = article_list[i:i + batch_size]
    page_info_dict = {}
    unable_to_get = {}
    await main(article_list1, page_info_dict, unable_to_get)
    
    # Save the results of this batch to a JSON file
    batch_result = {
        "batch_number": batch_number,
        "page_info_dict": page_info_dict,
        "unable_to_get": unable_to_get
    }
    results.append(batch_result)

    await asyncio.sleep(10) #to reduce the burden on the servers and get less failures

Now, lets see if me missed any articles

In [ ]:
#Check for any values that were not retreived
for batch_result in results:
    print(f"Batch {batch_result['batch_number']}:")
    print(f"page_info_dict length: {len(batch_result['page_info_dict'])}")
    print(f"unable_to_get length: {len(batch_result['unable_to_get'])}")

Batch 1:
page_info_dict length: 1000
unable_to_get length: 0
Batch 2:
page_info_dict length: 1000
unable_to_get length: 0
Batch 3:
page_info_dict length: 1000
unable_to_get length: 0
Batch 4:
page_info_dict length: 1000
unable_to_get length: 0
Batch 5:
page_info_dict length: 1000
unable_to_get length: 0
Batch 6:
page_info_dict length: 1000
unable_to_get length: 0
Batch 7:
page_info_dict length: 1000
unable_to_get length: 0
Batch 8:
page_info_dict length: 1000
unable_to_get length: 0
Batch 9:
page_info_dict length: 1000
unable_to_get length: 0
Batch 10:
page_info_dict length: 1000
unable_to_get length: 0
Batch 11:
page_info_dict length: 1000
unable_to_get length: 0
Batch 12:
page_info_dict length: 1000
unable_to_get length: 0
Batch 13:
page_info_dict length: 1000
unable_to_get length: 0
Batch 14:
page_info_dict length: 1000
unable_to_get length: 0
Batch 15:
page_info_dict length: 1000
unable_to_get length: 0
Batch 16:
page_info_dict length: 1000
unable_to_get length: 0
Batch 17:
page_in

Seems like the code worked as intended and processed all the articles. Let us store this to prevent any loss of data and avoiding rerunning of the code

In [ ]:
# Merge all page_info_dict dictionaries
all_page_info_dict = {}
for batch_result in results:
    all_page_info_dict.update(batch_result['page_info_dict'])

print(len(all_page_info_dict))

with open('data/all_page_info_dict.json', 'w') as json_file:
    json.dump(all_page_info_dict, json_file)

In [ ]:
# we can now merge this to the us_cities_by_state.
rev_id = pd.DataFrame(list(all_page_info_dict.items()), columns=['page_title', 'rev_id'])
us_cities_by_state = us_cities_by_state.merge(rev_id,on='page_title')

In [ ]:
#Check for nulls
us_cities_by_state.isna().sum()

state         0
page_title    0
url           0
rev_id        0
dtype: int64

We have successfully obtained the last revision id for all the articles. Let us save this dataframe to avoid running this code again.

In [ ]:
us_cities_by_state.to_csv('data/cleaned_us_cities_by_state_with_id.csv',index=False)

There seems to be no empty values, so it seems to be good for further processes

### Step 3: Getting the ORES scores

In [ ]:
us_cities_by_state = pd.read_csv('data/cleaned_us_cities_by_state_with_id.csv')

We can now work on obtaining the article quality scores for each article using the ORES_LEFTWING MODEL. You can read more about this here: [ORES LiftWing documentation](https://wikitech.wikimedia.org/wiki/Machine_Learning/LiftWing/Usage)

The following code example was heavily inspired from the code developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - August 15, 2023. You can look at this code in the helper_code/wp_ores_leftwing_example.ipynb. Also, please refer to this documentation to create API keys and check them.



In [ ]:
## CONSTANTS - API REQUIREMENTS
email = "<ENTER YOUR EMAIL HERE>"
access_token = "<ENTER YOUR ACCESS TOKEN HERE>"


In [ ]:
### Using async functions to generate multiple requests

API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"
API_LATENCY_ASSUMED = 0.002
API_THROTTLE_WAIT = (60.0 / 5000.0) - API_LATENCY_ASSUMED

REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<"+email+">, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer "+access_token
}

REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address': email,
    'access_token': access_token
}


ORES_REQUEST_DATA_TEMPLATE = {
    "lang": "en",
    "rev_id": "",
    "features": True
}


ores_scores = {}
no_prediction = []
key_error_list = []

article_id = {}

with open('data/all_page_info_dict.json', 'r') as file:
    article_id = json.load(file)

This code is also modified to work asyncronously like the previous one. However, note that we are rate limited to only 5000 requests per hour on the personal API option, here I have sliced the articles into batches of 5000 to make this run faster, and rerun this every hour or so using updated slicers to avoind getting the HTTP code:429 for rate limit exceeded. 

Please exercise caution while using the following code, as this might immediately exhause your quota for the hour. Repeated use of this code may lead to API restrictions/ban if misused.

In [ ]:
# create the list of articles to be processed.
article_list = list(us_cities_by_state.page_title)
article_list = article_list[21500:25000]

In [ ]:
# modified request_ores_score_per_article from helper_code/wp_ores_leftwing_example.ipynb to work async
async def request_ores_score_per_article(session, article_revid=None, email_address=None, access_token=None,
                                        endpoint_url=API_ORES_LIFTWING_ENDPOINT,
                                        model_name=API_ORES_EN_QUALITY_MODEL,
                                        request_data=ORES_REQUEST_DATA_TEMPLATE,
                                        header_format=REQUEST_HEADER_TEMPLATE,
                                        header_params=REQUEST_HEADER_PARAMS_TEMPLATE):
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token

    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")

    request_url = endpoint_url.format(model_name=model_name)

    headers = {}
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)

    try:
        print(request_data)
        async with session.post(request_url, headers=headers, data=json.dumps(request_data)) as response:
            json_response = await response.json()
        print(f"Task completed for rev_id {article_revid}")  # Add this line for debugging
        #print(json_response)
    except Exception as e:
        print(e)
        json_response = None
    return json_response

I have used a iteration of 1000, where the process_articles function saves the data into a json file for every 1000 successful iterations. You can modify this as you see fit, but this worked for my present data with no hiccups for 5000 articles.

In [ ]:
async def process_articles():
    async with aiohttp.ClientSession() as session:
        article_count = 0
        tasks = []
        articles_processed = 0  # Counter for articles processed
        articles_per_iteration = 1000 # Number of articles to process in each iteration

        for article in article_list:
            task = request_ores_score_per_article(session, article_revid=article_id[article],
                                                  email_address=email, access_token=access_token)
            tasks.append(task)

            # Increment the counter
            articles_processed += 1

            if articles_processed % articles_per_iteration == 0:
                # If 1000 articles have been processed, gather the results
                results = await asyncio.gather(*tasks)
                for article, score in zip(article_list[articles_processed - articles_per_iteration:articles_processed], results):
                    if score is None:
                        no_prediction.append(article)
                    else:
                        try:
                            print(article, score)
                            ores_scores[article] = score['enwiki']['scores'][str(article_id[article])]['articlequality']['score']['prediction']
                        except KeyError:
                            key_error_list.append(article)

                # Reset tasks for the next iteration
                tasks = []

                # Wait for 5 seconds
                time.sleep(5)

        # Save the final output to JSON
        ores_scores_json_object = json.dumps(ores_scores, indent=4)
        with open(f'ores_scores_final_6.json', 'w') as outfile:
            outfile.write(ores_scores_json_object)


In [ ]:
#Run this code to process for the sliced 5000 articles that are sliced
await process_articles()

We have successfully saved all the requested articles, and there seems to be no articles missed, which is great

### Step 4: Merging all the files

We can now merge all the quality scores from the data/ores folder to a single one

In [ ]:
import os
import json

directory_path = "data/ores_files"

# Initialize an empty dictionary for merging
merged_data = {}

# List JSON files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path, filename)
        
        # Load the contents of the JSON file
        with open(file_path, "r") as json_file:
            data = json.load(json_file)
            
        # Update the merged dictionary with the data
        merged_data.update(data)

# Save the merged data to a new JSON file
output_file_path = "data/merged_data.json"
with open(output_file_path, "w") as output_file:
    json.dump(merged_data, output_file, indent=4)

print(f"Merged data saved to {output_file_path}")


We have to merge merged_data_ores.json and the us_cities_by_state-cleaned to a new one

In [122]:
with open("data/merged_data_ores.json", 'r') as file:
    ores_final = json.load(file)

us_cities_by_state = pd.read_csv('data/cleaned_us_cities_by_state_with_id.csv')

In [123]:
print(us_cities_by_state.shape[0],len(ores_final))

21515 21515


Seems like we have captured the values for all the articles. Let us join these 2 datasets

In [124]:
# Merge on 'page_title'
us_cities_by_state['quality'] = us_cities_by_state['page_title'].map(ores_final)

In [125]:
us_cities_by_state

,state,page_title,url,rev_id,quality
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama",1171163550,C
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama",1177621427,C
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama",1168359898,C
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama",1165909508,GA
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama",1179139816,C
...,...,...,...,...,...
21510,Wyoming,"Wamsutter, Wyoming","https://en.wikipedia.org/wiki/Wamsutter,_Wyoming",1169591845,GA
21511,Wyoming,"Wheatland, Wyoming","https://en.wikipedia.org/wiki/Wheatland,_Wyoming",1176370621,GA
21512,Wyoming,"Worland, Wyoming","https://en.wikipedia.org/wiki/Worland,_Wyoming",1166347917,GA
21513,Wyoming,"Wright, Wyoming","https://en.wikipedia.org/wiki/Wright,_Wyoming",1166334449,GA


In [126]:
us_cities_by_state[us_cities_by_state['quality'].isna()]

,state,page_title,url,rev_id,quality


There seems to be no missing values, which validates our point from above. Let us now add the population data and the regions information for each state

In [127]:
# Load the US population data from an Excel file, skipping the first 4 rows of metadata.
us_population_data = pd.read_excel('data/NST-EST2022-POP.xlsx', skiprows=4)

# Remove unnecessary rows and reset the index.
us_population_data = us_population_data[4:]
us_population_data.reset_index(drop=True, inplace=True)

# Rename the columns for clarity.
us_population_data.columns = ['state', '2020_est', '2020', '2021', '2022']

# Filter and clean the data: Keep only rows where the 'state' names start with a period.
us_population_data = us_population_data[us_population_data['state'].str.contains('^\.', na=False)]
us_population_data['state'] = us_population_data['state'].str.slice(1)
us_population_data = us_population_data[['state', '2022']].reset_index(drop=True)

# Rename the columns to reflect the year.
us_population_data.columns = ['state', 'Population_2022']

#us_population_data


In [128]:
# Load regional division data from an Excel file
us_regions = pd.read_excel('data/US States by Region - US Census Bureau.xlsx')

# Fill missing REGION and DIVISION values with the previous valid value
us_regions['REGION'].ffill(inplace=True)
us_regions['DIVISION'].ffill(inplace=True)

# Filter rows with valid STATE entries (remove non-state rows)
us_regions = us_regions.dropna(subset=['STATE'])

# Convert column names to lowercase for consistency
us_regions.columns = us_regions.columns.str.lower()

#us_regions



In [129]:
us_cities_by_state['state'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Delaware', 'Florida', 'Georgia_(U.S._state)',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nevada', 'New_Hampshire', 'New_Jersey', 'New_Mexico', 'New_York',
       'North_Carolina', 'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode_Island', 'South_Carolina', 'South_Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West_Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

Looks like we need to clean some of the state names to merge with the other datasets

In [130]:
# Clean state names, handling any specific cases
us_cities_by_state['state'] = us_cities_by_state['state'].apply(lambda x: 'Georgia' if x == 'Georgia_(U.S._state)' else x)

# The cityxstate data had values like New_york, fixing that
us_cities_by_state['state'] = us_cities_by_state.state.str.replace('_', ' ')

In [143]:
# merge all the datasets
temp1 = us_regions.merge(us_population_data, on = 'state').merge(us_cities_by_state, on='state')
temp1.shape

(21515, 8)

The final dataset has taken shape. We can now clean up the names and save this dataframe for our analysis

In [144]:
temp1.drop(columns = ['url','region'], inplace = True)
renaming = {
    'division': 'regional_division',
    'Population_2022': 'population',
    'page_title':'article_title',
    'rev_id': 'revision_id',
    'quality':'article_quality'
}

temp1.rename(columns=renaming, inplace=True)
temp1.sort_values('article_title')

In [149]:
temp1.to_csv('data/wp_scored_city_articles_by_state.csv',index=False)